In [ ]:
#
# Prototyping parts of the following:
#
# Drawing Graphs to Convey Proximity: An Incremental Arrangement Method
# J.D. Cohen
# ACM Transactions on Computer-Human Interaction, Vol. 4, No. 3, September 1997, Pages 197–229.
#
import polars as pl
import networkx as nx

from math import log10, ceil
import time
import rtsvg
from rtsvg.polars_force_directed_layout import PolarsForceDirectedLayout
rt = rtsvg.RACETrack()

from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()

def graphAsDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)

#
# Attempting to match Figure 1 from "Drawing Graphs To Convey Proximity"
# - looks like i don't do enough iterations for proportional or semi-proportional
# - absolute stress looks like the papers version
#
_tiles_ = []
g       = _patterns_.createPattern('X')
df      = graphAsDataFrame(g)
for k in range(3):
    _lu_ = {'iteration':[], 'stress':[], 'trial':[], 'time':[]}
    for _trial_ in range(1): # should be 15 - 25
        t0    = time.time()
        pfdl  = rtsvg.PolarsForceDirectedLayout(g, k=k)
        t1    = time.time()
        _vec_ = pfdl.stressVector()
        for i in range(len(_vec_)):
            _lu_['iteration'].append(i), _lu_['stress'].append(log10(_vec_[i])), _lu_['trial'].append(_trial_), _lu_['time'].append(t1-t0)
    df_stress = pl.DataFrame(_lu_)
    _tiles_.append(rt.titleSVG(rt.xy(df_stress, x_field='iteration', y_field='stress', line_groupby_field='trial', dot_size=None), f'k={k}'))
#rt.tile(_tiles_, spacer=10) # uncomment to display

In [ ]:
def __getTargetDistances__(_g_): return dict(nx.all_pairs_dijkstra_path_length(_g_))
# Table IV of paper
def __everyNthMember__(Q, n): return [Q[i] for i in range(0, len(Q), n)]
def __disperseTheseVertices__(Q, increment_ratio=1):
    if len(Q) > increment_ratio + 1:
        F = __everyNthMember__(Q, 1+increment_ratio)
        B = Q - F
        F = __disperseTheseVertices__(F)
        return F.extend(B)
    else: return Q

def __orderVertices__(_g_, _dist_):
    Q = [n for n in nx.traversal.dfs_preorder_nodes(_g_)]
    return __disperseTheseVertices__(Q)

# Table III of paper
def __numberToAddThisTime__(_prev_, _final_, increment_ratio=1, increment_minimum=10):
    if _prev_ > 0: _inc_ = _prev_ * increment_ratio
    else:
        _inc_ = _final_
        while _inc_ > increment_minimum: _inc_ = ceil(_inc_ / (1 + increment_ratio))
    if _inc_ > _final_ - _prev_: _inc_ = _final_ - _prev_
    return _inc_

#
# Table II of paper
#
def arrangeIncrementally(g_connected, k=2.0):
    pos         = {}                                                  # Results
    V           = set(g_connected.nodes)
    _distances_ = __getTargetDistances__(g_connected)                 # Establish target distances
    Q           = __orderVertices__(g_connected, _distances_)         # Make vector Q of vertices in order of inclusion
    H           = set()                                               # Initialize vertices to arrange
    while H != V:
        for i in range(__numberToAddThisTime__(len(H), len(V))):
            v      = Q[0]                                             # Get next vertex
            h1     = __closestMember__(H,        v)                   # Find closest member of H
            h2     = __closestMember__(H - {h1}, v)                   # Next closest member of H
            pos[v] = __neighborlyLocation__(h1, h2)                   # Put new vertex near them
            Q      = Q[1:]                                            # This vertex is done
            H.add(v)
        _pos_ = __arrangeDirect__(H, pos, k=k)                        # Arrange accumulated subset
    return pos
